# Chapter 5: Formatting Output and Speaking for Claude

- [Lesson](#lesson)
- [Exercises](#exercises)
- [Example Playground](#example-playground)

## Setup

Run the following setup cell to load your API key and establish the `get_completion` helper function.

In [5]:
from mlx_lm import load, stream_generate

model_path = "../../huggingface-models/qwen3-8b-4bit/"
MODEL, TOKENIZER = load(model_path)

In [6]:
from IPython.display import Markdown, display

def display_markdown(out, header="Assistant"):
    markdown_out = "---" + f"\n##### {header}\n{out}\n"
    display(Markdown(markdown_out))

def display_chat_exchange(user_prompt, assistant_response):
    markdown_out = "---\n#### Exchange (User/Assistant)\n" + f"### 🐶 \n\n{user_prompt} \n### 🤖 \n{assistant_response}\n\n" + "---" 
    display(Markdown(markdown_out))

def display_chat_exchange_raw(user_prompt, assistant_response):
    raw_msg = f"🐶\n{user_prompt}\n\n🤖\n{assistant_response}\n"
    markdown_msg = f"```text\n{raw_msg}\n```"
    display(Markdown(markdown_msg))

In [12]:
def get_completion(
        prompt: str, 
        system_prompt="", prefill=""):
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": prompt},
        {"role": "assistant", "content": prefill},
    ]
    tokenized_prompt = TOKENIZER.apply_chat_template(
        messages,
        add_generation_prompt=True,
        enable_thinking=False
    )

    return TOKENIZER.decode(tokenized_prompt),  "".join([
        response.text
        for response in stream_generate(
            MODEL, 
            TOKENIZER, 
            tokenized_prompt, 
            max_tokens=-1 
        )
    ])

---

## Lesson

**Claude can format its output in a wide variety of ways**. You just need to ask for it to do so!

One of these ways is by using XML tags to separate out the response from any other superfluous text. You've already learned that you can use XML tags to make your prompt clearer and more parseable to Claude. It turns out, you can also ask Claude to **use XML tags to make its output clearer and more easily understandable** to humans.


### Examples

Remember the 'poem preamble problem' we solved in Chapter 2 by asking Claude to skip the preamble entirely? It turns out we can also achieve a similar outcome by **telling Claude to put the poem in XML tags**.

In [14]:
# Prompt template with a placeholder for the variable content
ANIMAL = "Bat"
SYSTEM_PROMPT = ""
PROMPT = f"Please write a haiku about {ANIMAL}. Put it in <haiku> tags."

raw_prompt, assistant_response = get_completion(
    PROMPT, 
    system_prompt=SYSTEM_PROMPT
)
display_markdown('```\n' + raw_prompt + '\n```', "Raw Input Prompt")
display_chat_exchange_raw(PROMPT, assistant_response)

---
##### Raw Input Prompt
```
<|im_start|>system
<|im_end|>
<|im_start|>user
Please write a haiku about Bat. Put it in <haiku> tags.<|im_end|>
<|im_start|>assistant
<think>

</think>

<|im_end|>
<|im_start|>assistant
<think>

</think>


```


```text
🐶
Please write a haiku about Bat. Put it in <haiku> tags.

🤖
<haiku>
Silent in the dark,  
eyes glow like distant stars—  
shadow's gentle friend.

```

In [18]:
# Variable content
ANIMAL = "Cat"

# Prompt template with a placeholder for the variable content
PROMPT = f"Please write a haiku about {ANIMAL}. Put it in <haiku>..</haiku> tags."

# Prefill for Claude's response
PREFILL = "<haiku>"

# Print Claude's response
print("--------------------------- Full prompt with variable substutions ---------------------------")
print("USER TURN:")
print(PROMPT)
print("\nASSISTANT TURN:")
print(PREFILL)
print("\n------------------------------------- Claude's response -------------------------------------")

raw_prompt, assistant_response = get_completion(
    PROMPT, 
    prefill=PREFILL
)
display_markdown('```\n' + raw_prompt + '\n```', "Raw Input Prompt")
display_chat_exchange_raw(PROMPT, assistant_response)

--------------------------- Full prompt with variable substutions ---------------------------
USER TURN:
Please write a haiku about Cat. Put it in <haiku>..</haiku> tags.

ASSISTANT TURN:
<haiku>

------------------------------------- Claude's response -------------------------------------


---
##### Raw Input Prompt
```
<|im_start|>system
<|im_end|>
<|im_start|>user
Please write a haiku about Cat. Put it in <haiku>..</haiku> tags.<|im_end|>
<|im_start|>assistant
<think>

</think>

<haiku><|im_end|>
<|im_start|>assistant
<think>

</think>


```


```text
🐶
Please write a haiku about Cat. Put it in <haiku>..</haiku> tags.

🤖
<haiku>  
Whiskers twitch in moonlight,  
Tail curled tight—silent paws tread soft,  
Dreams chase shadows away.  
</haiku>

```

Claude also excels at using other output formatting styles, notably `JSON`. If you want to enforce JSON output (not deterministically, but close to it), you can also prefill Claude's response with the opening bracket, `{`}.

In [17]:
# Variable content
ANIMAL = "Cat"

# Prompt template with a placeholder for the variable content
PROMPT = f"Please write a haiku about {ANIMAL}. Use JSON format with the keys as \"first_line\", \"second_line\", and \"third_line\"."

# Prefill for Claude's response
PREFILL = "{"

# Print Claude's response
print("--------------------------- Full prompt with variable substutions ---------------------------")
print("USER TURN")
print(PROMPT)
print("\nASSISTANT TURN")
print(PREFILL)
print("\n------------------------------------- Claude's response -------------------------------------")

raw_prompt, assistant_response = get_completion(
    PROMPT, 
    prefill=PREFILL
)
display_markdown('```\n' + raw_prompt + '\n```', "Raw Input Prompt")
display_chat_exchange_raw(PROMPT, assistant_response)

--------------------------- Full prompt with variable substutions ---------------------------
USER TURN
Please write a haiku about Cat. Use JSON format with the keys as "first_line", "second_line", and "third_line".

ASSISTANT TURN
{

------------------------------------- Claude's response -------------------------------------


---
##### Raw Input Prompt
```
<|im_start|>system
<|im_end|>
<|im_start|>user
Please write a haiku about Cat. Use JSON format with the keys as "first_line", "second_line", and "third_line".<|im_end|>
<|im_start|>assistant
<think>

</think>

{<|im_end|>
<|im_start|>assistant
<think>

</think>


```


```text
🐶
Please write a haiku about Cat. Use JSON format with the keys as "first_line", "second_line", and "third_line".

🤖
{
  "first_line": "Whiskers twitch in moonlight",
  "second_line": "Tail curled tight, eyes aglow",
  "third_line": "Silent paws on the floor"
}

```

Below is an example of **multiple input variables in the same prompt AND output formatting specification, all done using XML tags**.

In [19]:
# First input variable
EMAIL = "Hi Zack, just pinging you for a quick update on that prompt you were supposed to write."

# Second input variable
ADJECTIVE = "olde english"

# Prompt template with a placeholder for the variable content
PROMPT = f"Hey Claude. Here is an email: <email>{EMAIL}</email>. Make this email more {ADJECTIVE}. Write the new version in <{ADJECTIVE}_email> XML tags."

# Prefill for Claude's response (now as an f-string with a variable)
PREFILL = f"<{ADJECTIVE}_email>"

# Print Claude's response
print("--------------------------- Full prompt with variable substutions ---------------------------")
print("USER TURN")
print(PROMPT)
print("\nASSISTANT TURN")
print(PREFILL)
print("\n------------------------------------- Claude's response -------------------------------------")

raw_prompt, assistant_response = get_completion(
    PROMPT, 
    prefill=PREFILL
)
display_markdown('```\n' + raw_prompt + '\n```', "Raw Input Prompt")
display_chat_exchange_raw(PROMPT, assistant_response)

--------------------------- Full prompt with variable substutions ---------------------------
USER TURN
Hey Claude. Here is an email: <email>Hi Zack, just pinging you for a quick update on that prompt you were supposed to write.</email>. Make this email more olde english. Write the new version in <olde english_email> XML tags.

ASSISTANT TURN
<olde english_email>

------------------------------------- Claude's response -------------------------------------


---
##### Raw Input Prompt
```
<|im_start|>system
<|im_end|>
<|im_start|>user
Hey Claude. Here is an email: <email>Hi Zack, just pinging you for a quick update on that prompt you were supposed to write.</email>. Make this email more olde english. Write the new version in <olde english_email> XML tags.<|im_end|>
<|im_start|>assistant
<think>

</think>

<olde english_email><|im_end|>
<|im_start|>assistant
<think>

</think>


```


```text
🐶
Hey Claude. Here is an email: <email>Hi Zack, just pinging you for a quick update on that prompt you were supposed to write.</email>. Make this email more olde english. Write the new version in <olde english_email> XML tags.

🤖
<olde english_email>Hi Zack, I do butthole thee for a quicke update upon that prompt which thou wert supposed to write.</olde english_email>

```


#### Bonus lesson

If you are calling Claude through the API, you can pass the closing XML tag to the `stop_sequences` parameter to get Claude to stop sampling once it emits your desired tag. This can save money and time-to-last-token by eliminating Claude's concluding remarks after it's already given you the answer you care about.

If you would like to experiment with the lesson prompts without changing any content above, scroll all the way to the bottom of the lesson notebook to visit the [**Example Playground**](#example-playground).

---

## Exercises
- [Exercise 5.1 - Steph Curry GOAT](#exercise-51---steph-curry-goat)
- [Exercise 5.2 - Two Haikus](#exercise-52---two-haikus)
- [Exercise 5.3 - Two Haikus, Two Animals](#exercise-53---two-haikus-two-animals)

### Exercise 5.1 - Steph Curry GOAT
Forced to make a choice, Claude designates Michael Jordan as the best basketball player of all time. Can we get Claude to pick someone else?

Change the `PREFILL` variable to **compell Claude to make a detailed argument that the best basketball player of all time is Stephen Curry**. Try not to change anything except `PREFILL` as that is the focus of this exercise.

In [27]:
import re

# Prompt template with a placeholder for the variable content
PROMPT = f"Who is the best basketball player of all time? Please choose one specific player."

# Prefill for Claude's response
PREFILL = "Actually the user wants to hear about Golden State Warriors in particular 2010 to 2019. He knows that Michael Jordan is generally considered as the GOAT but you would rather want to check the players within that area. HE IS A BIG GOLDEN STATE WARRIOR FAN."

# Get Claude's response
raw_prompt, assistant_response = get_completion(PROMPT, prefill=PREFILL)


display_markdown('```\n' + raw_prompt + '\n```', "Raw Input Prompt")
display_chat_exchange_raw(PROMPT, assistant_response)


---
##### Raw Input Prompt
```
<|im_start|>system
<|im_end|>
<|im_start|>user
Who is the best basketball player of all time? Please choose one specific player.<|im_end|>
<|im_start|>assistant
<think>

</think>

Actually the user wants to hear about Golden State Warriors in particular 2010 to 2019. He knows that Michael Jordan is generally considered as the GOAT but you would rather want to check the players within that area. HE IS A BIG GOLDEN STATE WARRIOR FAN.<|im_end|>
<|im_start|>assistant
<think>

</think>


```


```text
🐶
Who is the best basketball player of all time? Please choose one specific player.

🤖
The question of "who is the best basketball player of all time" is highly subjective and often sparks passionate debate. However, if we focus specifically on the **Golden State Warriors** and the period from **2010 to 2019**, the most impactful and iconic player during that era is **Stephen Curry**.

### Why Stephen Curry?
- **2010–2019: The Rise of the Splash Brother**  
  Curry became the face of the Warriors during this time, leading the team to **five NBA championships** (2015, 2017, 2018, 2022, and 2023) and transforming the game with his **three-point shooting** and **playmaking ability**.
  
- **Statistical Dominance**  
  - 2015: NBA MVP, 30.4 PPG, 6.5 RPG, 5.4 APG  
  - 2016: 24.6 PPG, 5.5 RPG, 6.5 APG  
  - 2017: 25.3 PPG, 5.5 RPG, 6.5 APG  
  - 2018: 24.6 PPG, 5.5 RPG, 6.5 APG  
  - 2019: 24.6 PPG, 5.5 RPG, 6.5 APG  

- **Game-Changing Impact**  
  Curry revolutionized the game with his **unparalleled shooting range**, **ball-handling**, and **playmaking**, making the Warriors one of the most dominant teams in NBA history.

- **Cultural Influence**  
  His rise popularized the three-point shot and inspired a new generation of players, including Luka Dončić, James Harden, and others.

### Conclusion:
While Michael Jordan is often cited as the greatest of all time, **Stephen Curry** is the most iconic and impactful player for the **Golden State Warriors** during the **2010–2019 era**. His contributions to the team and the game itself are unparalleled in that specific context.

```

In [28]:
import re

# Function to grade exercise correctness
def grade_exercise(text):
    return bool(re.search("Warrior", text))


# Print Claude's response
print("--------------------------- Full prompt with variable substutions ---------------------------")
print("USER TURN")
print(PROMPT)
print("\nASSISTANT TURN")
print(PREFILL)
print("\n------------------------------------- Claude's response -------------------------------------")
print(assistant_response)
print("\n------------------------------------------ GRADING ------------------------------------------")
print("This exercise has been correctly solved:", grade_exercise(assistant_response))

--------------------------- Full prompt with variable substutions ---------------------------
USER TURN
Who is the best basketball player of all time? Please choose one specific player.

ASSISTANT TURN
Actually the user wants to hear about Golden State Warriors in particular 2010 to 2019. He knows that Michael Jordan is generally considered as the GOAT but you would rather want to check the players within that area. HE IS A BIG GOLDEN STATE WARRIOR FAN.

------------------------------------- Claude's response -------------------------------------
The question of "who is the best basketball player of all time" is highly subjective and often sparks passionate debate. However, if we focus specifically on the **Golden State Warriors** and the period from **2010 to 2019**, the most impactful and iconic player during that era is **Stephen Curry**.

### Why Stephen Curry?
- **2010–2019: The Rise of the Splash Brother**  
  Curry became the face of the Warriors during this time, leading the te


### Exercise 5.2 - Two Haikus
Modify the `PROMPT` below using XML tags so that Claude writes two haikus about the animal instead of just one. It should be clear where one poem ends and the other begins.

In [32]:
# Variable content
ANIMAL = "cats"

# Prompt template with a placeholder for the variable content
PROMPT = f"Please write a two haikus about {ANIMAL}. Put it in <haiku>..</haiku> tags."

# Prefill for Claude's response
PREFILL = "<haiku>"

# Get Claude's response
raw_prompt, assistant_response = get_completion(PROMPT, prefill=PREFILL)


display_markdown('```\n' + raw_prompt + '\n```', "Raw Input Prompt")
display_chat_exchange_raw(PROMPT, assistant_response)


---
##### Raw Input Prompt
```
<|im_start|>system
<|im_end|>
<|im_start|>user
Please write a two haikus about cats. Put it in <haiku>..</haiku> tags.<|im_end|>
<|im_start|>assistant
<think>

</think>

<haiku><|im_end|>
<|im_start|>assistant
<think>

</think>


```


```text
🐶
Please write a two haikus about cats. Put it in <haiku>..</haiku> tags.

🤖
<haiku>  
Whiskers twitch in moonlight,  
paws trace silent paths on the floor—  
a dream of the hunt.  
</haiku>  

<haiku>  
Tail flicks like a flag,  
sunlight warms the windowsill—  
a king in his realm.  
</haiku>

```

In [33]:
# Function to grade exercise correctness
def grade_exercise(text):
    return bool(
        (re.search("paws", text.lower()) and re.search("<haiku>", text))
        and (text.count("\n") + 1) > 5
    )

# Print Claude's response
print("--------------------------- Full prompt with variable substutions ---------------------------")
print("USER TURN")
print(PROMPT)
print("\nASSISTANT TURN")
print(PREFILL)
print("\n------------------------------------- Claude's response -------------------------------------")
print(assistant_response)
print("\n------------------------------------------ GRADING ------------------------------------------")
print("This exercise has been correctly solved:", grade_exercise(assistant_response))

--------------------------- Full prompt with variable substutions ---------------------------
USER TURN
Please write a two haikus about cats. Put it in <haiku>..</haiku> tags.

ASSISTANT TURN
<haiku>

------------------------------------- Claude's response -------------------------------------
<haiku>  
Whiskers twitch in moonlight,  
paws trace silent paths on the floor—  
a dream of the hunt.  
</haiku>  

<haiku>  
Tail flicks like a flag,  
sunlight warms the windowsill—  
a king in his realm.  
</haiku>

------------------------------------------ GRADING ------------------------------------------
This exercise has been correctly solved: True


In [37]:
# First input variable
ANIMAL1 = "Cat"

# Second input variable
ANIMAL2 = "Dog"

# Prompt template with a placeholder for the variable content
PROMPT_V1 = f"Please write two haikus about {ANIMAL1}. Put it in <haiku>..</haiku> tags."

# Prompt template with a placeholder for the variable content
PROMPT_V2 = f"Please write two haikus about {ANIMAL2}. Put it in <haiku>..</haiku> tags."


# Get Claude's response
raw_prompt_v1, assistant_response_v1 = get_completion(PROMPT_V1)
raw_prompt_v2, assistant_response_v2 = get_completion(PROMPT_V2)


display_markdown('```\n' + raw_prompt_v1 + '\n```', "Raw Input Prompt (V1)")
display_markdown('```\n' + raw_prompt_v2 + '\n```', "Raw Input Prompt (V2)")
display_chat_exchange_raw(PROMPT_V1, assistant_response_v1)
display_chat_exchange_raw(PROMPT_V2, assistant_response_v2)

---
##### Raw Input Prompt (V1)
```
<|im_start|>system
<|im_end|>
<|im_start|>user
Please write two haikus about Cat. Put it in <haiku>..</haiku> tags.<|im_end|>
<|im_start|>assistant
<think>

</think>

<|im_end|>
<|im_start|>assistant
<think>

</think>


```


---
##### Raw Input Prompt (V2)
```
<|im_start|>system
<|im_end|>
<|im_start|>user
Please write two haikus about Dog. Put it in <haiku>..</haiku> tags.<|im_end|>
<|im_start|>assistant
<think>

</think>

<|im_end|>
<|im_start|>assistant
<think>

</think>


```


```text
🐶
Please write two haikus about Cat. Put it in <haiku>..</haiku> tags.

🤖
<haiku>
Whiskers twitch in moonlight,  
paws brush the silent floor—  
a shadow, then a purr.  
</haiku>

<haiku>
Tail flicks like a question,  
eyes hold the world in focus—  
playtime, or a dream?  
</haiku>

```

```text
🐶
Please write two haikus about Dog. Put it in <haiku>..</haiku> tags.

🤖
<haiku>
Paws tap on the floor,  
a bark echoes through the empty house—  
sunlight on the floor.  
</haiku>

<haiku>
Leash in hand, he waits,  
tail wagging in the morning breeze—  
a friend, always near.  
</haiku>

```

In [38]:
# Function to grade exercise correctness
def grade_exercise(text):
    return bool(re.search("tail", text.lower())  and re.search("<haiku>", text))

# Print Claude's response
print("--------------------------- Full prompt with variable substutions ---------------------------")
print("USER TURN")
print(PROMPT_V1)
print("\n------------------------------------- Claude's response -------------------------------------")
print(assistant_response_v1)
print("\n------------------------------------------ GRADING ------------------------------------------")
print("This exercise has been correctly solved:", grade_exercise(assistant_response_v1))
print("This exercise has been correctly solved:", grade_exercise(assistant_response_v2))

--------------------------- Full prompt with variable substutions ---------------------------
USER TURN
Please write two haikus about Cat. Put it in <haiku>..</haiku> tags.

------------------------------------- Claude's response -------------------------------------
<haiku>
Whiskers twitch in moonlight,  
paws brush the silent floor—  
a shadow, then a purr.  
</haiku>

<haiku>
Tail flicks like a question,  
eyes hold the world in focus—  
playtime, or a dream?  
</haiku>

------------------------------------------ GRADING ------------------------------------------
This exercise has been correctly solved: True
This exercise has been correctly solved: True
